# Search Contribution Overview

Use this notebook to track whether search is growing as a share of site traffic.

Business questions:
- Are search clicks and organic sessions trending up?
- Is search share of total sessions improving?
- Are there recent drops that need investigation?

In [1]:
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../..")

import plotly.express as px

import lifeline_theme
from lla_data import config
from lla_data.bq import default_query_window, get_client, run_query

lifeline_theme.inject_fonts()

DAYS_BACK = config.DEFAULT_DAYS_BACK
window = default_query_window(days_back=DAYS_BACK)
client = get_client()

print(f"Project: {config.PROJECT_ID}")
print(f"Datasets: GA4={config.GA4_DATASET}, GSC={config.SEARCHCONSOLE_DATASET}")
print(f"Date range: {window.start_date} to {window.end_date}")

Project: lifeline-website-480522
Datasets: GA4=analytics_315584957, GSC=searchconsole
Date range: 2026-01-26 to 2026-02-23


In [2]:
from lla_data.bq import build_date_params

query = f"""
SELECT
  report_date,
  SUM(gsc_clicks) AS gsc_clicks,
  SUM(gsc_impressions) AS gsc_impressions,
  SUM(organic_sessions) AS organic_sessions,
  SUM(total_sessions) AS total_sessions,
  SAFE_DIVIDE(SUM(organic_sessions), NULLIF(SUM(total_sessions), 0)) AS search_share
FROM `{config.PROJECT_ID}.{config.SEARCHCONSOLE_DATASET}.seo_page_daily`
WHERE report_date BETWEEN DATE(@start_date) AND DATE(@end_date)
GROUP BY report_date
ORDER BY report_date
"""

df = run_query(client, query, params=build_date_params(window))
df.tail()

/home/aido/projects/lla-data/.venv/lib/python3.14/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,report_date,gsc_clicks,gsc_impressions,organic_sessions,total_sessions,search_share
23,2026-02-18,0,0,3579,9826,0.364238
24,2026-02-19,2403,279983,3987,10527,0.378740
25,2026-02-20,1827,280993,3163,9207,0.343543
26,2026-02-21,0,0,2596,7248,0.358168
27,2026-02-22,0,0,2944,7975,0.369154


In [3]:
fig_clicks = px.line(
    df,
    x="report_date",
    y=["gsc_clicks", "organic_sessions"],
    template="lifeline",
    title="Search Clicks vs Organic Sessions",
)
lifeline_theme.add_lifeline_logo(fig_clicks)
fig_clicks.show()

fig_share = px.line(
    df,
    x="report_date",
    y="search_share",
    template="lifeline",
    title="Search Share of Sessions",
)
fig_share.update_yaxes(tickformat=".0%")
lifeline_theme.add_lifeline_logo(fig_share)
fig_share.show()